## Import libraries

In [1]:
!pip install lightfm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 8.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp310-cp310-linux_x86_64.whl size=806104 sha256=531a091bb73e28b65e909973af4935c5d48052dbbf82840ea02f438a111dab0d
  Stored in directory: /root/.cache/pip/wheels/4f/9b/7e/0b256f2168511d8fa4dae4fae0200fdbd729eb424a912ad636
Successfully built lightfm


In [2]:
import numpy as np
import pandas as pd
from collections import Counter, defaultdict
from sklearn.preprocessing import normalize
from scipy.sparse import coo_matrix, csr_matrix
from sklearn.model_selection import train_test_split
from lightfm import LightFM
from lightfm.evaluation import precision_at_k, recall_at_k, auc_score
import kagglehub

In [3]:
from lightfm.data import Dataset

In [4]:
from transformers import SiglipModel, SiglipProcessor
from transformers import CLIPProcessor, CLIPModel
from transformers import BlipProcessor, BlipModel
from transformers import ViltProcessor, ViltModel
from sklearn.preprocessing import normalize
from collections import Counter
import torch
import kagglehub
import requests
import os
from PIL import Image
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.preprocessing import normalize
from lightfm import LightFM
from lightfm.evaluation import precision_at_k, recall_at_k, auc_score
from collections import defaultdict
import numpy as np
import torch.nn as nn
from scipy.sparse import lil_matrix

## Load and filter dataset

In [5]:
path = kagglehub.dataset_download("threnjen/board-games-database-from-boardgamegeek/versions/4")
print("Path to dataset files:", path)

100%|██████████| 148M/148M [00:01<00:00, 78.4MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/threnjen/board-games-database-from-boardgamegeek/versions/4


In [6]:
%cd /root/.cache/kagglehub/datasets/threnjen/board-games-database-from-boardgamegeek/versions/4

/root/.cache/kagglehub/datasets/threnjen/board-games-database-from-boardgamegeek/versions/4


In [7]:
games_df = pd.read_csv("games.csv")
user_ratings_df = pd.read_csv("user_ratings.csv")

In [8]:
games_df_filtered = games_df[["Description", "ImagePath"]].dropna()
texts = games_df_filtered["Description"].tolist()
image_paths = games_df_filtered["ImagePath"].tolist()

print("Ejemplo de descripciones:")
print(texts[:5])
print("\nEjemplo de rutas de imágenes:")
print(image_paths[:5])

Ejemplo de descripciones:
['die macher game seven sequential political race different region germany player charge national political party manage limited resource help party victory win party victory point regional election different way score victory point regional election supply eighty victory point depend size region party second party win regional election medium influence region party receive mediacontrol victory point party national party membership grow game progress supply fair number victory point lastly party score victory point party platform match national opinion end gamethe   edition feature party old west germany support   player   edition support player reunite germany update feature rule     edition support player add short fiveround variant additional rule update original designer', 'dragonmaster tricktaking card game base old game call coup dtat player give supply plastic gem represent point player dealer different hand slightly different goal hand card deal dealer

In [9]:
def load_images_from_urls(image_urls):
    images = []
    for url in image_urls:
        response = requests.get(url, stream=True)
        if response.status_code == 200:
            image = Image.open(response.raw).convert("RGB")
            images.append(image)
        else:
            print(f"Failed to load image from: {url}")
    return images

num_samples = 100
images = load_images_from_urls(image_paths[:num_samples])

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:1054: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


## Generate Siglip embeddings

In [19]:
model_name = "google/siglip-base-patch16-224"
model = SiglipModel.from_pretrained(model_name)
processor = SiglipProcessor.from_pretrained(model_name)

config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/813M [00:00<?, ?B/s]

In [ ]:
def generate_siglip_embeddings(texts, images):
    text_embeds = []
    image_embeds = []
    multimodal_embeds = []

    for text, image in zip(texts, images):
        inputs = processor(
            text=texts,
            images=images,
            return_tensors="pt",
            padding="max_length",
            truncation=True,
            max_length=64
        )

        with torch.no_grad():
            outputs = model(**inputs)

        text_embeds.append(outputs.text_embeds[0])
        image_embeds.append(outputs.image_embeds[0])
        multimodal_embeds.append(outputs.text_embeds[0] + outputs.image_embeds[0])

    return torch.stack(text_embeds), torch.stack(image_embeds), torch.stack(multimodal_embeds)

siglip_text_embeds, siglip_image_embeds, siglip_multimodal_embeds = generate_siglip_embeddings(texts[:num_samples], images)

print(f"Embeddings de texto: {siglip_text_embeds.shape}")
print(f"Embeddings de imágenes: {siglip_image_embeds.shape}")
print(f"Embeddings multimodales: {siglip_multimodal_embeds.shape}")

In [ ]:
import torch

# Save embeddings to files
torch.save(siglip_text_embeds, "SIGLIP_text_embeddings.pt")
torch.save(siglip_image_embeds, "SIGLIP_image_embeddings.pt")
torch.save(siglip_multimodal_embeds, "SIGLIP_multimodal_embeddings.pt")

# Optionally, move files to Google Drive for persistence
from google.colab import drive
drive.mount('/content/drive')
!cp text_embeddings.pt /content/drive/MyDrive/
!cp image_embeddings.pt /content/drive/MyDrive/
!cp multimodal_embeddings.pt /content/drive/MyDrive/


## Bibliografía

[Siglip - HuggingFace](https://huggingface.co/docs/transformers/model_doc/siglip)